In [ ]:
%pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 976.9/976.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import torch
torch.cuda.is_available()
!nvidia-smi  # Show GPU details

Mon Mar 31 08:15:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Extract Dataset ZIP Folder**

In [ ]:
!gdown 1i5ou5RgqB8ae1YDGZpq9VujM-W_XazGg

Downloading...
From (original): https://drive.google.com/uc?id=1i5ou5RgqB8ae1YDGZpq9VujM-W_XazGg
From (redirected): https://drive.google.com/uc?id=1i5ou5RgqB8ae1YDGZpq9VujM-W_XazGg&confirm=t&uuid=9f59cbb3-1ed8-46e0-b58b-4724cbaed8c4
To: /content/toyota_models.zip
100% 387M/387M [00:07<00:00, 49.9MB/s]


In [ ]:
# prompt: extract toyota_models.zip to datasets folder

import zipfile
import os

# Specify the zip file path and extraction directory
zip_file_path = 'toyota_models.zip'
extract_dir = 'datasets'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Optional: Print a message to confirm extraction
print(f"Successfully extracted {zip_file_path} to {extract_dir}")


Successfully extracted toyota_models.zip to datasets


In [ ]:
import os
import random
import shutil

def split_data(source_dir, train_dir, val_dir, test_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """Splits data into training, validation, and testing sets while preserving folder structure."""

    for root, dirs, files in os.walk(source_dir):
        rel_path = os.path.relpath(root, source_dir)  # Get relative path from source
        train_subdir = os.path.join(train_dir, rel_path)
        val_subdir = os.path.join(val_dir, rel_path)
        test_subdir = os.path.join(test_dir, rel_path)

        if not files:
            continue  # Skip empty directories

        os.makedirs(train_subdir, exist_ok=True)
        os.makedirs(val_subdir, exist_ok=True)
        os.makedirs(test_subdir, exist_ok=True)

        for file in files:
            src_file = os.path.join(root, file)
            rand_val = random.random()

            if rand_val < train_ratio:
                dest_file = os.path.join(train_subdir, file)
            elif rand_val < train_ratio + val_ratio:
                dest_file = os.path.join(val_subdir, file)
            else:
                dest_file = os.path.join(test_subdir, file)

            shutil.copy2(src_file, dest_file)  # Preserve metadata when copying

# Example usage:
source_directory = 'datasets/toyota models'  # Replace with your source directory
train_directory = 'datasets/train'
validation_directory = 'datasets/validation'
test_directory = 'datasets/test'

split_data(source_directory, train_directory, validation_directory, test_directory)

In [ ]:
# prompt: shortest way possible, delete folder datasets and content

import shutil

shutil.rmtree('datasets/toyota models', ignore_errors=True)


In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8 classification model
model = YOLO('yolov8n-cls.pt')  # You can also try 'yolov8m-cls.pt' for a larger model

# Train the model
model.train(
    data="datasets",  # Path to dataset folder (no YAML needed)
    imgsz=224,  # Image size for classification
    batch=16,  # Batch size (increase if you have more VRAM)
    epochs=100,  # Number of training epochs
    device="cuda",  # Use GPU if available
    optimizer="AdamW",  # Optimized weight updates
    lr0=0.001,  # Initial learning rate
    lrf=0.01,  # Final learning rate (OneCycleLR policy)
    weight_decay=0.0005,  # Regularization to prevent overfitting
    dropout=0.3,  # Add dropout for regularization
    workers=4,  # Number of workers for data loading
    mixup=0.2,  # MixUp augmentation
    fliplr=0.5,  # Horizontal flip probability
    flipud=0.2,  # Vertical flip probability
)


100%|██████████| 5.31M/5.31M [00:00<00:00, 84.8MB/s]


Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=datasets, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=cuda, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

100%|██████████| 5.35M/5.35M [00:00<00:00, 94.7MB/s]


AMP: checks passed ✅


train: Scanning /content/datasets/train... 1921 images, 0 corrupt: 100%|██████████| 1921/1921 [00:00<00:00, 3594.04it/s]

...

                   all      0.891      0.982



100 epochs completed in 0.856 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,450,252 parameters, 0 gradients, 3.3 GFLOPs
train: /content/datasets/train... found 1921 images in 12 classes ✅ 
val: /content/datasets/val... found 394 images in 12 classes ✅ 
test: /content/datasets/test... found 418 images in 12 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 13/13 [00:03<00:00,  3.69it/s]


                   all      0.901      0.985
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e45178c2950>
curves: []
curves_results: []
fitness: 0.942893385887146
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9010152220726013, 'metrics/accuracy_top5': 0.9847715497016907, 'fitness': 0.942893385887146}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.0956611269035411, 'inference': 0.30194070304764514, 'loss': 0.0003840050743895005, 'postprocess': 0.0005681142115209524}
task: 'classify'
top1: 0.9010152220726013
top5: 0.9847715497016907